<a name="top"></a>
# Enigma machine

This is an implementation of an Enigma machine in Python. See below for links that describe the Enigma machine and how it was used.

The Enigma machine has a bunch of components which can be swapped or modified to change its behaviour. The components are:

* The plugboard
* The wheels (three chosen from a set of five)
* The reflector (two available, though generally not swapped)

The plugboard and wheel selection were changed every day. The wheel orientation was changed every message.

## Design sketch
Each of the components can be thought of as a "letter transformer", which take a letter and input and give a different letter as output. From a given setup (plugboard setting or wheel orientation), these transformations are deterministic: if nothing moves, the same letter input will give the same letter output. Depending on the component, forward and backward transformations can be different. For instance, the wheel I converts `a` to `e` forward, and `a` to `u` backward.

This means we can take an object oriented approach to building the Enigma machine. The machine itself is a collection (aggregation) of components. Each component keeps track of its current state. 

The components have an inheritance hierarchy.

* [LetterTransformer](#lettertransformer)
  * [Plugobard](#plugboard)
    * [Reflector](#reflector)
  * [SimpleWheel](#simplewheel)
    * [Wheel](#wheel)

The `LetterTransformer` is the base class and defines the basic operations all the transformers apply. 

A `Plugboard` is a type of `LetterTransformer` that swaps only some letters, and acts the same both forward and backward. The `Reflector` acts like a `Plugboard` but with 13 pairs of swaps.

A `SimpleWheel` has different forward and backward transforms, and also rotates. A `Wheel` is the same, but the indicator "ring" around the outside can be rotated around the core. This ring of a `Wheel` has a notch that can control when other `SimpleWheel`s and `Wheel`s are rotated in the Enigma.

Note that all the logic of when the wheels rotate is controlled by the Enigma machine.

* [Engima](#enigma)
* [Testing Enigma](#testingenigma)

### Implmentation note
The normal way to define a class in Python is to define all the methods, class variables, and instance variables at the same time. However, that makes it difficult to place the discussion of the various methods and what they do near the definitions. 

This notebook takes a variant approach of defining the base class, then defining methods in separate cells and adding them to the class with the `setattr()` procedure.


## See also
* Specification from [Codes and Ciphers](http://www.codesandciphers.org.uk/enigma/rotorspec.htm) page.

* Example Enigma machines from [Louise Dale](http://enigma.louisedade.co.uk/enigma.html) (full simulation) and [EnigmaCo](http://enigmaco.de/enigma/enigma.html) (good animation of the wheels, but no ring settings).

* There's also the nice Enigma simulator for Android by [Franklin Heath](https://franklinheath.co.uk/2012/02/04/our-first-app-published-enigma-simulator/), available on the [Google Play store](https://play.google.com/store/apps/details?id=uk.co.franklinheath.enigmasim&hl=en_GB).

* Enigma wiring from the [Crypto Museum](http://www.cryptomuseum.com/crypto/enigma/wiring.htm).

First, some general-purpose and utility imports. 

`pos` and `unpos` convert between letters and numbers (in range 0-25 inclusive).

In [108]:
import string
import collections

cat = ''.join

def clean(text): return cat(l.lower() for l in text if l in string.ascii_letters)

def pos(letter): 
    if letter in string.ascii_lowercase:
        return ord(letter) - ord('a')
    elif letter in string.ascii_uppercase:
        return ord(letter) - ord('A')
    else:
        return ''
    
def unpos(number): return chr(number % 26 + ord('a'))

The wheel specifications show what positions `a` to `z` (ignoring the ring) go to. For instance, Wheel 1 converts `a` to `e` forward, and `a` to `u` backward. The notch positions show where the wheel advance notches are on the wheel rings. The reflector specifications show the reflected pairs.

In [109]:
wheel_i_spec = 'ekmflgdqvzntowyhxuspaibrcj'
wheel_ii_spec = 'ajdksiruxblhwtmcqgznpyfvoe'
wheel_iii_spec = 'bdfhjlcprtxvznyeiwgakmusqo'
wheel_iv_spec = 'esovpzjayquirhxlnftgkdcmwb'
wheel_v_spec = 'vzbrgityupsdnhlxawmjqofeck'
wheel_vi_spec = 'jpgvoumfyqbenhzrdkasxlictw'
wheel_vii_spec = 'nzjhgrcxmyswboufaivlpekqdt'
wheel_viii_spec = 'fkqhtlxocbjspdzramewniuygv'
beta_wheel_spec = 'leyjvcnixwpbqmdrtakzgfuhos'
gamma_wheel_spec = 'fsokanuerhmbtiycwlqpzxvgjd'

wheel_i_notches = ['q']
wheel_ii_notches = ['e']
wheel_iii_notches = ['v']
wheel_iv_notches = ['j']
wheel_v_notches = ['z']
wheel_vi_notches = ['z', 'm']
wheel_vii_notches = ['z', 'm']
wheel_viii_notches = ['z', 'm']

reflector_b_spec = 'ay br cu dh eq fs gl ip jx kn mo tz vw'
reflector_c_spec = 'af bv cp dj ei go hy kr lz mx nw tq su'

In [110]:
# class LetterTransformer(object):
#     def __init__(self, specification, raw_transform=False):
#         if raw_transform:
#             transform = specification
#         else:
#             transform = self.parse_specification(specification)
#         self.validate_transform(transform)
#         self.make_transform_map(transform)
    
#     def parse_specification(self, specification):
#         return list(zip(string.ascii_lowercase, clean(specification)))
#         # return specification
    
#     def validate_transform(self, transform):
#         """A set of pairs, of from-to"""
#         if len(transform) != 26:
#             raise ValueError("Transform specification has {} pairs, requires 26".
#                 format(len(transform)))
#         for p in transform:
#             if len(p) != 2:
#                 raise ValueError("Not all mappings in transform "
#                     "have two elements")
#         if len(set([p[0] for p in transform])) != 26:
#             raise ValueError("Transform specification must list 26 origin letters") 
#         if len(set([p[1] for p in transform])) != 26:
#             raise ValueError("Transform specification must list 26 destination letters") 

#     def make_empty_transform(self):
#         self.forward_map = [0] * 26
#         self.backward_map = [0] * 26
            
#     def make_transform_map(self, transform):
#         self.make_empty_transform()
#         for p in transform:
#             self.forward_map[pos(p[0])] = pos(p[1])
#             self.backward_map[pos(p[1])] = pos(p[0])
#         return self.forward_map, self.backward_map
    
#     def forward(self, letter):
#         if letter in string.ascii_lowercase:
#             return unpos(self.forward_map[pos(letter)])
#         else:
#             return ''
                
#     def backward(self, letter):
#         if letter in string.ascii_lowercase:
#             return unpos(self.backward_map[pos(letter)])
#         else:
#             return ''

<a name="lettertransformer"></a>
# Letter transformer
[Top](#top)

A generic transformer of letters. All components in the Enigma are based on this. 

The transformer has two directions, `forward` and `backward`. In each direction, a given letter is transformed into a different letter. Both transformations are general permutations of the alphabet (i.e. each letter goes to one and only one new letter). There is no general requirement for the `forward` and `backward` transformations to have any particular relationship to each other (even though most do in the Enigma machine).

When created, it must be given the transformation which should be applied. A raw transform is a sequence of letter pairs, such that `p[0]` is transformed to `p[1]` forwards, and `p[1]` goes to `p[0]` backwards.

If the transform is not raw, it's assumed that the specification is a sequence of the `p[1]`s, and the standard alphabet gives the `p[0]`s.

In [111]:
class LetterTransformer(object):
    def __init__(self, specification, raw_transform=False):
        if raw_transform:
            transform = specification
        else:
            transform = self.parse_specification(specification)
        self.validate_transform(transform)
        self.make_transform_map(transform)

Parse a specification: convert a string of destination letters into a list of pairs. 

In [112]:
def parse_specification(self, specification):
    return list(zip(string.ascii_lowercase, clean(specification)))
    # return specification

setattr(LetterTransformer, "parse_specification", parse_specification)

In [113]:
parse_specification(None, wheel_i_spec)

[('a', 'e'),
 ('b', 'k'),
 ('c', 'm'),
 ('d', 'f'),
 ('e', 'l'),
 ('f', 'g'),
 ('g', 'd'),
 ('h', 'q'),
 ('i', 'v'),
 ('j', 'z'),
 ('k', 'n'),
 ('l', 't'),
 ('m', 'o'),
 ('n', 'w'),
 ('o', 'y'),
 ('p', 'h'),
 ('q', 'x'),
 ('r', 'u'),
 ('s', 's'),
 ('t', 'p'),
 ('u', 'a'),
 ('v', 'i'),
 ('w', 'b'),
 ('x', 'r'),
 ('y', 'c'),
 ('z', 'j')]

Checks that a transform is valid.

In [114]:
def validate_transform(self, transform):
    """A set of pairs, of from-to"""
    if len(transform) != 26:
        raise ValueError("Transform specification has {} pairs, requires 26".
            format(len(transform)))
    for p in transform:
        if len(p) != 2:
            raise ValueError("Not all mappings in transform "
                "have two elements")
    if len(set([p[0] for p in transform])) != 26:
        raise ValueError("Transform specification must list 26 origin letters") 
    if len(set([p[1] for p in transform])) != 26:
        raise ValueError("Transform specification must list 26 destination letters") 

setattr(LetterTransformer, "validate_transform", validate_transform)        

The empty transform maps each letter to itself, forward and backward. A useful starting point for creating the maps needed.

The forward and backward maps are `list`s of numbers (rather than `dict`s of letters to letters) to make the calculations easier when it comes to the wheels, and wheels with turnable indicator rings.

In [115]:
# def make_empty_transform(self):
#     self.forward_map = [0] * 26
#     self.backward_map = [0] * 26

# setattr(LetterTransformer, "make_empty_transform", make_empty_transform)    

In [116]:
def make_empty_transform(self):
    self.forward_map = list(range(26))
    self.backward_map = list(range(26))

setattr(LetterTransformer, "make_empty_transform", make_empty_transform)    

Make the transform. Starting from an empty transform, mutate it to include the swaps. Note that the forward and backward swaps are stored separately. 

In [117]:
def make_transform_map(self, transform):
    self.make_empty_transform()
    for p in transform:
        self.forward_map[pos(p[0])] = pos(p[1])
        self.backward_map[pos(p[1])] = pos(p[0])
    return self.forward_map, self.backward_map

setattr(LetterTransformer, "make_transform_map", make_transform_map)

In [118]:
def forward(self, letter):
    if letter in string.ascii_lowercase:
        return unpos(self.forward_map[pos(letter)])
    else:
        return ''

def backward(self, letter):
    if letter in string.ascii_lowercase:
        return unpos(self.backward_map[pos(letter)])
    else:
        return ''

setattr(LetterTransformer, "forward", forward)
setattr(LetterTransformer, "backward", backward)    

In [119]:
tmap = [('z', 'a')] + [(l, string.ascii_lowercase[i+1]) for i, l in enumerate(string.ascii_lowercase[:-1])]
tmap

[('z', 'a'),
 ('a', 'b'),
 ('b', 'c'),
 ('c', 'd'),
 ('d', 'e'),
 ('e', 'f'),
 ('f', 'g'),
 ('g', 'h'),
 ('h', 'i'),
 ('i', 'j'),
 ('j', 'k'),
 ('k', 'l'),
 ('l', 'm'),
 ('m', 'n'),
 ('n', 'o'),
 ('o', 'p'),
 ('p', 'q'),
 ('q', 'r'),
 ('r', 's'),
 ('s', 't'),
 ('t', 'u'),
 ('u', 'v'),
 ('v', 'w'),
 ('w', 'x'),
 ('x', 'y'),
 ('y', 'z')]

In [120]:
cat(collections.OrderedDict.fromkeys('zyxwc' + string.ascii_lowercase))

'zyxwcabdefghijklmnopqrstuv'

In [121]:
tmap2 = list(zip(string.ascii_lowercase, cat(collections.OrderedDict.fromkeys('zyxwc' + string.ascii_lowercase))))

In [122]:
lt = LetterTransformer(tmap, raw_transform = True)
assert(lt.forward_map == [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 0])
assert(lt.backward_map == [25, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])
lt.forward_map, lt.backward_map

([1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  0],
 [25,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24])

In [123]:
lt = LetterTransformer(cat(collections.OrderedDict.fromkeys('zyxwc' + string.ascii_lowercase)))
assert(lt.forward_map == [25, 24, 23, 22, 2, 0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])
assert(lt.backward_map == [5, 6, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 3, 2, 1, 0])
assert(cat(lt.forward(l) for l in string.ascii_lowercase) == 'zyxwcabdefghijklmnopqrstuv')
assert(cat(lt.backward(l) for l in string.ascii_lowercase) == 'fgehijklmnopqrstuvwxyzdcba')
lt.forward_map, lt.backward_map

([25,
  24,
  23,
  22,
  2,
  0,
  1,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21],
 [5,
  6,
  4,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  3,
  2,
  1,
  0])

In [124]:
cat(lt.forward(l) for l in string.ascii_lowercase)

'zyxwcabdefghijklmnopqrstuv'

In [125]:
cat(lt.backward(l) for l in string.ascii_lowercase)

'fgehijklmnopqrstuvwxyzdcba'

<a name="plugboard"></a>
## Plugboard
[Top](#top)

A `Plugboard` is a `LetterTransformer` that swaps some pairs of letters, and does the same swaps forward and backward.

In [126]:
class Plugboard(LetterTransformer):
    def parse_specification(self, specification):
        return [tuple(clean(p)) for p in specification.split()]
    
    def validate_transform(self, transform):
        """A set of pairs, of from-to"""
        for p in transform:
            if len(p) != 2:
                raise ValueError("Not all mappings in transform"
                    "have two elements")
    
#     def make_empty_transform(self):
#         self.forward_map = list(range(26))
#         self.backward_map = list(range(26))
        
    def make_transform_map(self, transform):
        expanded_transform = transform + [tuple(reversed(p)) for p in transform]
        return super(Plugboard, self).make_transform_map(expanded_transform)

In [127]:
pb = Plugboard([('a', 'z'), ('b', 'y')], raw_transform=True)

In [128]:
cat(pb.forward(l) for l in string.ascii_lowercase)

'zycdefghijklmnopqrstuvwxba'

In [129]:
cat(pb.backward(l) for l in string.ascii_lowercase)

'zycdefghijklmnopqrstuvwxba'

In [130]:
pb = Plugboard('az by')

In [131]:
cat(pb.forward(l) for l in string.ascii_lowercase), cat(pb.backward(l) for l in string.ascii_lowercase)

('zycdefghijklmnopqrstuvwxba', 'zycdefghijklmnopqrstuvwxba')

In [132]:
pb = Plugboard('ua pf rq so ni ey bg hl tx zj'.upper())
assert(pb.forward_map == pb.backward_map)
assert(pb.forward_map == [20, 6, 2, 3, 24, 15, 1, 11, 13, 25, 10, 7, 12, 8, 18, 5, 17, 16, 14, 23, 0, 21, 22, 19, 4, 9])
assert(cat(pb.forward(l) for l in string.ascii_lowercase) == 'ugcdypblnzkhmisfrqoxavwtej')
assert(cat(pb.backward(l) for l in string.ascii_lowercase) == 'ugcdypblnzkhmisfrqoxavwtej')
cat(pb.forward(l) for l in string.ascii_lowercase), cat(pb.backward(l) for l in string.ascii_lowercase)

('ugcdypblnzkhmisfrqoxavwtej', 'ugcdypblnzkhmisfrqoxavwtej')

<a name="reflector"></a>
## Reflector
[Top](#top)

A `Reflector` is a `Plugboard` that takes exactly 13 pairs of letters to swap.

In [133]:
class Reflector(Plugboard):
    def validate_transform(self, transform):
        if len(transform) != 13:
            raise ValueError("Reflector specification has {} pairs, requires 13".
                format(len(transform)))
        if len(set([p[0] for p in transform] + 
                    [p[1] for p in transform])) != 26:
            raise ValueError("Reflector specification does not contain 26 letters")
        try:
            super(Reflector, self).validate_transform(transform)
        except ValueError as v:
            raise ValueError("Not all mappings in reflector have two elements")

In [134]:
# reflector_b_text = '(AY) (BR) (CU) (DH) (EQ) (FS) (GL) (IP) (JX) (KN) (MO) (TZ) (VW)'
reflector_b_l = [tuple(clean(p)) for p in reflector_b_spec.split()]
reflector_b_l

[('a', 'y'),
 ('b', 'r'),
 ('c', 'u'),
 ('d', 'h'),
 ('e', 'q'),
 ('f', 's'),
 ('g', 'l'),
 ('i', 'p'),
 ('j', 'x'),
 ('k', 'n'),
 ('m', 'o'),
 ('t', 'z'),
 ('v', 'w')]

In [135]:
reflector_b = Reflector(reflector_b_spec)
assert(reflector_b.forward_map == reflector_b.backward_map)
assert(reflector_b.forward_map == [24, 17, 20, 7, 16, 18, 11, 3, 15, 23, 13, 6, 14, 10, 12, 8, 4, 1, 5, 25, 2, 22, 21, 9, 0, 19])
assert(cat(reflector_b.forward(l) for l in string.ascii_lowercase) == 'yruhqsldpxngokmiebfzcwvjat')
assert(cat(reflector_b.backward(l) for l in string.ascii_lowercase) == 'yruhqsldpxngokmiebfzcwvjat')

In [136]:
cat(reflector_b.forward(l) for l in string.ascii_lowercase)

'yruhqsldpxngokmiebfzcwvjat'

In [137]:
reflector_c = Reflector(reflector_c_spec)

In [138]:
cat(reflector_c.forward(l) for l in string.ascii_lowercase)

'fvpjiaoyedrzxwgctkuqsbnmhl'

<a name="simplewheel"></a>
## SimpleWheel
[Top](#top)

A `SimpleWheel` has different forward and backward maps, and also a position. The position is set with the `set_position` method (and initially in the creator), and the wheel can advance using the `advance` method. 

How the position is used is best explained with an example. The Enigma wheel 1, in the neutral position, transforms `a` to `e` (+4 letters) and `b` to `k` (+10 letters). When the wheel is in position `b` and an `a` in enciphered, it's the _second_ element of the map that's used, so `a` would be advanced 10 letters, to give `j`.

This means that when using the letter transformation maps, you use the element in the map that's offset by the position of the wheel. When enciphering a `c`, you'd normally use transformation at position 2 in the map; if the wheel is in position 7, you'd instead use the transform at position 2 + 7 = 9 in the map.

There are various modulus operators to keep the numbers in the requried range, meaning you can wrap around the map and around the wheel.

Note the use of `__getattribute__` to give a more human-friendly version of the position without making it a method call. That allows you to write `wheel.position` and `wheel.position_l` and get the appropriate answers.

In [139]:
class SimpleWheel(LetterTransformer):
    def __init__(self, transform, position='a', raw_transform=False):
        super(SimpleWheel, self).__init__(transform, raw_transform)
        self.set_position(position)
        
    def __getattribute__(self, name):
        if name=='position_l':
            return unpos(self.position)
        else:
            return object.__getattribute__(self, name)  

Set the wheel to a new position. Note that it expects a letter, not a number.

In [140]:
def set_position(self, position):
    self.position = ord(position) - ord('a')
        
setattr(SimpleWheel, 'set_position', set_position)       

Advance the wheel one step. Note that advancing beyond position 25 moves back to 0.

In [141]:
def advance(self):
    self.position = (self.position + 1) % 26
    return self.position

setattr(SimpleWheel, 'advance', advance)    

Do the encipherment forward and backward. Note how the map element to use is affected by the wheel position, and how the modulus wraps that map element around the wheel if needed.

In [142]:
def forward(self, letter):
    if letter in string.ascii_lowercase:
        return unpos((self.forward_map[(pos(letter) + self.position) % 26] - self.position))
    else:
        return ''

def backward(self, letter):
    if letter in string.ascii_lowercase:
        return unpos((self.backward_map[(pos(letter) + self.position) % 26] - self.position))
    else:
        return ''
        
setattr(SimpleWheel, 'forward', forward)        
setattr(SimpleWheel, 'backward', backward)  

In [143]:
rotor_1_transform = list(zip(string.ascii_lowercase, 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'.lower()))
rotor_1_transform

[('a', 'e'),
 ('b', 'k'),
 ('c', 'm'),
 ('d', 'f'),
 ('e', 'l'),
 ('f', 'g'),
 ('g', 'd'),
 ('h', 'q'),
 ('i', 'v'),
 ('j', 'z'),
 ('k', 'n'),
 ('l', 't'),
 ('m', 'o'),
 ('n', 'w'),
 ('o', 'y'),
 ('p', 'h'),
 ('q', 'x'),
 ('r', 'u'),
 ('s', 's'),
 ('t', 'p'),
 ('u', 'a'),
 ('v', 'i'),
 ('w', 'b'),
 ('x', 'r'),
 ('y', 'c'),
 ('z', 'j')]

In [144]:
rotor_1_transform = list(zip(string.ascii_lowercase, 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'.lower()))
wheel_1 = SimpleWheel(rotor_1_transform, raw_transform=True)
assert(cat(wheel_1.forward(l) for l in string.ascii_lowercase) == 'ekmflgdqvzntowyhxuspaibrcj')
assert(cat(wheel_1.backward(l) for l in string.ascii_lowercase) == 'uwygadfpvzbeckmthxslrinqoj')

In [145]:
wheel_1.forward_map

[4,
 10,
 12,
 5,
 11,
 6,
 3,
 16,
 21,
 25,
 13,
 19,
 14,
 22,
 24,
 7,
 23,
 20,
 18,
 15,
 0,
 8,
 1,
 17,
 2,
 9]

In [146]:
wheel_1.advance()
wheel_1.forward('a')

'j'

In [147]:
cat(wheel_1.forward(l) for l in string.ascii_lowercase), cat(wheel_1.backward(l) for l in string.ascii_lowercase)

('jlekfcpuymsnvxgwtrozhaqbid', 'vxfzceouyadbjlsgwrkqhmpnit')

In [148]:
wheel_1.position_l

'b'

In [149]:
wheel_2 = SimpleWheel(wheel_ii_spec)
assert(cat(wheel_2.forward(l) for l in string.ascii_lowercase) == 'ajdksiruxblhwtmcqgznpyfvoe')
assert(cat(wheel_2.backward(l) for l in string.ascii_lowercase) == 'ajpczwrlfbdkotyuqgenhxmivs')

In [150]:
cat(wheel_2.forward(l) for l in string.ascii_lowercase), cat(wheel_2.backward(l) for l in string.ascii_lowercase)

('ajdksiruxblhwtmcqgznpyfvoe', 'ajpczwrlfbdkotyuqgenhxmivs')

In [151]:
wheel_3 = SimpleWheel(wheel_iii_spec)
wheel_3.set_position('a')
wheel_3.advance()
assert(cat(wheel_3.forward(l) for l in string.ascii_lowercase) == 'cegikboqswuymxdhvfzjltrpna')
assert(cat(wheel_3.backward(l) for l in string.ascii_lowercase) == 'zfaobrcpdteumygxhwivkqjnls')
assert(wheel_3.position == 1)
assert(wheel_3.position_l == 'b')

for _ in range(24): wheel_3.advance()
assert(wheel_3.position == 25)
assert(wheel_3.position_l == 'z')
assert(cat(wheel_3.forward(l) for l in string.ascii_lowercase) == 'pcegikmdqsuywaozfjxhblnvtr')
assert(cat(wheel_3.backward(l) for l in string.ascii_lowercase) == 'nubhcqdterfvgwoaizjykxmslp')

wheel_3.advance()
assert(wheel_3.position == 0)
assert(wheel_3.position_l == 'a')
assert(cat(wheel_3.forward(l) for l in string.ascii_lowercase) == 'bdfhjlcprtxvznyeiwgakmusqo')
assert(cat(wheel_3.backward(l) for l in string.ascii_lowercase) == 'tagbpcsdqeufvnzhyixjwlrkom')

cat(wheel_3.forward(l) for l in string.ascii_lowercase), cat(wheel_3.backward(l) for l in string.ascii_lowercase)

('bdfhjlcprtxvznyeiwgakmusqo', 'tagbpcsdqeufvnzhyixjwlrkom')

<a name="wheel"></a>
## Wheel
[Top](#top)

This is the same as a `SimpleWheel`, but with the addition of a ring.

The ring is moveable around the core of the wheel (with the wiring). This means that moving the ring changes the orientation of the core and wiring for the same setting.

| Wheel with notch | Notch showing peg to hold it in place |
| ---------------- | ------------------------------------- |
| <img src="enigma-notch.jpg" alt="Enigma wheel with notch" width=300> | <img src="dtu_notch_big.jpg" alt="Enigma wheel with notch" width=300> |
| (From [Crypto museum](http://www.cryptomuseum.com/crypto/enigma/img/300879/035/full.jpg)) | From [Matematik Sider](http://www.matematiksider.dk/enigma/dtu_notch_big.jpg) |

Though it's not very visible in the right hand image, the extra metal below the ring shows a spring-loaded peg on the wheel core which drops into the ring, with one hole per letter. The ring setting is where the peg drops into the ring.

The notch setting is used in the full Enigma to control when the wheels step forward.

Note that the constructor calls the superclass's constructor, then sets the positions properly with a call to `set_position`.

In [152]:
class Wheel(SimpleWheel):
    def __init__(self, transform, ring_notch_letters, ring_setting=1, position='a', raw_transform=False):
        self.ring_notch_letters = ring_notch_letters
        self.ring_setting = ring_setting
        super(Wheel, self).__init__(transform, position=position, raw_transform=raw_transform)
        self.set_position(position)

The `position` of the wheel is the orientation of the core. It's the same as the ring if the ring setting is 1. The `position_l` attribute is used to report the position of the ring letter, which is what the Enigma operator would see. 

In [153]:
def __getattribute__(self,name):
    if name=='position_l':
        return unpos(self.position + self.ring_setting - 1)
    else:
        return object.__getattribute__(self, name)

def set_position(self, position):
#     self.position = (pos(position) - self.ring_setting + 1) % 26
    if isinstance(position, str):
        self.position = (pos(position) - self.ring_setting + 1) % 26
    else:
        self.position = (position - self.ring_setting) % 26
#     self.notch_positions = [(pos(position) - pos(p)) % 26  for p in self.ring_notch_letters]
    self.notch_positions = [(self.position + self.ring_setting - 1 - pos(p)) % 26  for p in self.ring_notch_letters]
        
setattr(Wheel, '__getattribute__', __getattribute__)        
setattr(Wheel, 'set_position', set_position)    

Advance the wheel. Again, note the superclass call, followed by the update of the notch positions.

In [154]:
def advance(self):
        super(Wheel, self).advance()
        self.notch_positions = [(p + 1) % 26 for p in self.notch_positions]
        # self.position_l = unpos(self.position + self.ring_setting - 1)
        return self.position
    
setattr(Wheel, 'advance', advance)    

In [155]:
wheel_3 = Wheel(wheel_iii_spec, wheel_iii_notches, position='b', ring_setting=1)

In [156]:
wheel_3.position, wheel_3.notch_positions

(1, [6])

In [157]:
wheel_6 = Wheel(wheel_vi_spec, wheel_vi_notches, position='b', ring_setting=3)
wheel_6.position, wheel_6.notch_positions

(25, [2, 15])

In [158]:
for _ in range(27):
    wheel_6.advance()
    print(wheel_6.position, wheel_6.notch_positions)

0 [3, 16]
1 [4, 17]
2 [5, 18]
3 [6, 19]
4 [7, 20]
5 [8, 21]
6 [9, 22]
7 [10, 23]
8 [11, 24]
9 [12, 25]
10 [13, 0]
11 [14, 1]
12 [15, 2]
13 [16, 3]
14 [17, 4]
15 [18, 5]
16 [19, 6]
17 [20, 7]
18 [21, 8]
19 [22, 9]
20 [23, 10]
21 [24, 11]
22 [25, 12]
23 [0, 13]
24 [1, 14]
25 [2, 15]
0 [3, 16]


In [159]:
wheel = Wheel(wheel_iii_spec, wheel_iii_notches, position='b', 
            ring_setting=3)
wheel.set_position(12)
assert(wheel.position == 9)
assert(16 in wheel.notch_positions)
assert(wheel.position_l =='l')

In [160]:
wheel.notch_positions

[16]

In [161]:
wheel.position_l

'l'

In [162]:
wheel_3 = Wheel(wheel_iii_spec, wheel_iii_notches, position='b', ring_setting=1)
assert(wheel_3.position == 1)
assert(wheel_3.notch_positions == [6])
assert(wheel_3.position_l == 'b')
wheel_3.advance()
assert(wheel_3.position == 2)
assert(wheel_3.notch_positions == [7])
assert(wheel_3.position_l == 'c')

In [163]:
wheel_6 = Wheel(wheel_vi_spec, wheel_vi_notches, position='b', ring_setting=3)
wheel_6.notch_positions

[2, 15]

In [164]:
wheel_6 = Wheel(wheel_vi_spec, wheel_vi_notches, position='b', ring_setting=3)
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'xkqhwpvngzrcfoiaselbtymjdu')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'ptlyrmidoxbswhnfckquzgeavj')
assert(wheel_6.position == 25)
assert(2 in wheel_6.notch_positions)
assert(15 in wheel_6.notch_positions)
assert(wheel_6.position_l == 'b')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'jpgvoumfyqbenhzrdkasxlictw')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'skxqlhcnwarvgmebjptyfdzuio')
assert(wheel_6.position == 0)
assert(3 in wheel_6.notch_positions)
assert(16 in wheel_6.notch_positions)
assert(wheel_6.position_l == 'c')

for _ in range(22): wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'mgxantkzsyqjcufirldvhoewbp')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'dymswobuplgraevzkqifntxcjh')
assert(wheel_6.position == 22)
assert(25 in wheel_6.notch_positions)
assert(12 in wheel_6.notch_positions)
assert(wheel_6.position_l == 'y')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'fwzmsjyrxpibtehqkcugndvaol')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'xlrvnatokfqzduyjphemswbigc')
assert(wheel_6.position == 23)
assert(0 in wheel_6.notch_positions)
assert(13 in wheel_6.notch_positions)
assert(wheel_6.position_l == 'z')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'vylrixqwohasdgpjbtfmcuznke')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'kqumzsnjepyctxiogdlrvahfbw')
assert(wheel_6.position == 24)
assert(1 in wheel_6.notch_positions)
assert(14 in wheel_6.notch_positions)
assert(wheel_6.position_l == 'a')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'xkqhwpvngzrcfoiaselbtymjdu')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'ptlyrmidoxbswhnfckquzgeavj')
assert(wheel_6.position == 25)
assert(2 in wheel_6.notch_positions)
assert(15 in wheel_6.notch_positions)
assert(wheel_6.position_l == 'b')

wheel_6.advance()
assert(cat(wheel_6.forward(l) for l in string.ascii_lowercase) == 'jpgvoumfyqbenhzrdkasxlictw')
assert(cat(wheel_6.backward(l) for l in string.ascii_lowercase) == 'skxqlhcnwarvgmebjptyfdzuio')
assert(wheel_6.position == 0)
assert(3 in wheel_6.notch_positions)
assert(16 in wheel_6.notch_positions)
assert(wheel_6.position_l == 'c')

In [165]:
wheel_6.position, wheel_6.position_l, wheel_6.notch_positions

(0, 'c', [3, 16])

<a name="enigma"></a>
## Enigma
[Top](#top)

This is the full Enigma machine.

It's a collection of the various components defined above. There are three wheels (left, middle, and right), a plugboard, and a reflector.

The `__getattribute__` method returns the state of the machine in friendly form, generally by asking the components to return the relevant attributes.

In [167]:
class Enigma(object):
    def __init__(self, reflector_spec,
                 left_wheel_spec, left_wheel_notches,
                 middle_wheel_spec, middle_wheel_notches,
                 right_wheel_spec, right_wheel_notches,
                 left_ring_setting, middle_ring_setting, right_ring_setting,
                 plugboard_setting):
        self.reflector = Reflector(reflector_spec)
        self.left_wheel = Wheel(left_wheel_spec, left_wheel_notches, ring_setting=left_ring_setting)
        self.middle_wheel = Wheel(middle_wheel_spec, middle_wheel_notches, ring_setting=middle_ring_setting)
        self.right_wheel = Wheel(right_wheel_spec, right_wheel_notches, ring_setting=right_ring_setting)
        self.plugboard = Plugboard(plugboard_setting)
        
    def __getattribute__(self,name):
        if name=='wheel_positions':
            return self.left_wheel.position, self.middle_wheel.position, self.right_wheel.position 
        elif name=='wheel_positions_l':
            return self.left_wheel.position_l, self.middle_wheel.position_l, self.right_wheel.position_l 
        elif name=='notch_positions':
            return (self.left_wheel.notch_positions, 
                    self.middle_wheel.notch_positions, 
                    self.right_wheel.notch_positions)
        else:
            return object.__getattribute__(self, name)

Set the wheels to the initial positions. 

In [168]:
def set_wheels(self, left_wheel_position, middle_wheel_position, right_wheel_position):
    self.left_wheel.set_position(left_wheel_position)
    self.middle_wheel.set_position(middle_wheel_position)
    self.right_wheel.set_position(right_wheel_position)

setattr(Enigma, 'set_wheels', set_wheels)

`lookup` just follows a path through the machine without changing the positions of any parts. It just follows a signal from the input, thorough all the components, to the reflector, back through all the components, to the output.

In [169]:
def lookup(self, letter):
    a = self.plugboard.forward(letter)
    b = self.right_wheel.forward(a)
    c = self.middle_wheel.forward(b)
    d = self.left_wheel.forward(c)
    e = self.reflector.forward(d)
    f = self.left_wheel.backward(e)
    g = self.middle_wheel.backward(f)
    h = self.right_wheel.backward(g)
    i = self.plugboard.backward(h)
    return i

setattr(Enigma, 'lookup', lookup)

`advance` moves the wheels on one step. The right wheel always advances. If the notch is in the zero position, the wheel also advances the wheel to the left. 

It follows the 'double stepping' behaviour of the engima machines.

In [170]:
def advance(self):
    advance_middle = False
    advance_left = False
    if 0 in self.right_wheel.notch_positions:
        advance_middle = True
    if 0 in self.middle_wheel.notch_positions:
        advance_left = True
        advance_middle = True
    self.right_wheel.advance()
    if advance_middle: self.middle_wheel.advance()
    if advance_left: self.left_wheel.advance()

setattr(Enigma, 'advance', advance)        

Finally, encipher letters and messages. 

Note that the wheels advance _before_ the letter signal is sent through the machine: in the physical machine, the advancing is done by pressing the key on the keyboard. 

Also note that the messages are cleaned before use, so letters are converted to lower case and non-letters are removed.

In [220]:
def encipher_letter(self, letter):
    self.advance()
    return self.lookup(letter)

def encipher(self, message, debug=False):
    enciphered = ''
    for letter in clean(message):
        enciphered += self.encipher_letter(letter)
        if debug:
            print('Wheels now', list(self.wheel_positions_l), 'enciphering {} -> {}'.format(letter, self.lookup(letter)))
    return enciphered

setattr(Enigma, 'encipher_letter', encipher_letter)
setattr(Enigma, 'encipher', encipher)
setattr(Enigma, 'decipher', encipher)

<a name="testingenigma"></a>
## Testing Enigma
[Top](#top)

Some tests of the Enigma machine.

In [172]:
enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_notches,
                wheel_ii_spec, wheel_ii_notches,
                wheel_iii_spec, wheel_iii_notches,
                1, 1, 1,
                '')

In [173]:
enigma.lookup('a')

'u'

In [174]:
enigma.lookup('u')

'a'

In [175]:
cat(enigma.lookup(l) for l in string.ascii_lowercase)

'uejobtpzwcnsrkdgvmlfaqiyxh'

In [176]:
cat(enigma.lookup(enigma.lookup(l)) for l in string.ascii_lowercase)

'abcdefghijklmnopqrstuvwxyz'

In [177]:
assert(cat(enigma.lookup(enigma.lookup(l)) for l in string.ascii_lowercase) == string.ascii_lowercase)

In [178]:
# check the middle wheel turns over
enigma.set_wheels('a', 'a', 'a')
for i in range(26):
    print(i, '::', 
          enigma.left_wheel.position_l, enigma.middle_wheel.position_l, enigma.right_wheel.position_l, ';',
          enigma.left_wheel.notch_positions, enigma.middle_wheel.notch_positions, enigma.right_wheel.notch_positions, 
         cat(enigma.lookup(l) for l in string.ascii_lowercase))
    enigma.advance()

0 :: a a a ; [10] [22] [5] uejobtpzwcnsrkdgvmlfaqiyxh
1 :: a a b ; [10] [22] [6] baqmfexihswpdytlcvjozrkgnu
2 :: a a c ; [10] [22] [7] djralkwpobfeyqihncxzvugsmt
3 :: a a d ; [10] [22] [8] zlejcuitgdmbkonsvxphfqyrwa
4 :: a a e ; [10] [22] [9] gcblwtakqzhdosmxiunfryepvj
5 :: a a f ; [10] [22] [10] osnirgfmdpvuhcajwebxlkqtzy
6 :: a a g ; [10] [22] [11] wymvnqzjlhoicekuftxrpdasbg
7 :: a a h ; [10] [22] [12] cjafkdztpbeuormiwnvhlsqyxg
8 :: a a i ; [10] [22] [13] xijuyslvbczgnmqwotfrdhpaek
9 :: a a j ; [10] [22] [14] lfzrwbytjisaovmuxdkhpneqgc
10 :: a a k ; [10] [22] [15] tkezcqynuwbpvhslfxoaimjrgd
11 :: a a l ; [10] [22] [16] kiwfnduxbsaotelqpvjmgrchzy
12 :: a a m ; [10] [22] [17] sfkutbpoxycrnmhgwlaedzqijv
13 :: a a n ; [10] [22] [18] baqwlkhgrsfextpocijnvudmzy
14 :: a a o ; [10] [22] [19] teofbdzxqkjyrscvimnawpuhlg
15 :: a a p ; [10] [22] [20] mhypswrbzxqvaondkgeutlfjci
16 :: a a q ; [10] [22] [21] cpasnrhgkuixzevbyfdwjotlqm
17 :: a a r ; [10] [22] [22] dlfatcjwygvbnmzrxpueskhqio
18 :: a

Formal test of middle wheel turnover

In [179]:
enigma.set_wheels('a', 'a', 't')
assert(enigma.wheel_positions == (0, 0, 19))
assert(cat(enigma.wheel_positions_l) == 'aat')
assert(enigma.notch_positions == ([10], [22], [24]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'puvioztjdhxmlyeawsrgbcqknf')

enigma.advance()
assert(enigma.wheel_positions == (0, 0, 20))
assert(cat(enigma.wheel_positions_l) == 'aau')
assert(enigma.notch_positions == ([10], [22], [25]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'baigpldqcowfyzjehvtsxrkumn')

enigma.advance()
assert(enigma.wheel_positions == (0, 0, 21))
assert(cat(enigma.wheel_positions_l) == 'aav')
assert(enigma.notch_positions == ([10], [22], [0]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'mnvfydiwgzsoablrxpkutchqej')

enigma.advance()
assert(enigma.wheel_positions == (0, 1, 22))
assert(cat(enigma.wheel_positions_l) == 'abw')
assert(enigma.notch_positions == ([10], [23], [1]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ulfopcykswhbzvderqixanjtgm')

enigma.advance()
assert(enigma.wheel_positions == (0, 1, 23))
assert(cat(enigma.wheel_positions_l) == 'abx')
assert(enigma.notch_positions == ([10], [23], [2]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'qmwftdyovursbzhxaklejicpgn')

enigma.advance()
assert(enigma.wheel_positions == (0, 1, 24))
assert(cat(enigma.wheel_positions_l) == 'aby')
assert(enigma.notch_positions == ([10], [23 ], [3]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'oljmzxrvucybdqasngpwihtfke')


Test of middle wheel advancing the left wheel, exhibiting the "double step" behaviour.

In [180]:
enigma.set_wheels('a', 'd', 't')
assert(enigma.wheel_positions == (0, 3, 19))
assert(cat(enigma.wheel_positions_l) == 'adt')
assert(enigma.notch_positions == ([10], [25], [24]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'zcbpqxwsjiuonmldethrkygfva')

enigma.advance()
assert(enigma.wheel_positions == (0, 3, 20))
assert(cat(enigma.wheel_positions_l) == 'adu')
assert(enigma.notch_positions == ([10], [25], [25]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ehprawjbngotxikcsdqlzyfmvu')

enigma.advance()
assert(enigma.wheel_positions == (0, 3, 21))
assert(cat(enigma.wheel_positions_l) == 'adv')
assert(enigma.notch_positions == ([10], [25], [0]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'eqzxarpihmnvjkwgbfuyslodtc')

enigma.advance()
assert(enigma.wheel_positions == (0, 4, 22))
assert(cat(enigma.wheel_positions_l) == 'aew')
assert(enigma.notch_positions == ([10], [0], [1]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'qedcbtpluzmhkongavwfirsyxj')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 23))
assert(cat(enigma.wheel_positions_l) == 'bfx')
assert(enigma.notch_positions == ([11], [1], [2]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'iwuedhsfazqxytvrkpgncoblmj')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 24))
assert(cat(enigma.wheel_positions_l) == 'bfy')
assert(enigma.notch_positions == ([11], [1], [3]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'baknstqzrmcxjdvygiefwoulph')

print(enigma.wheel_positions, enigma.wheel_positions_l, enigma.notch_positions, 
         cat(enigma.lookup(l) for l in string.ascii_lowercase))


(1, 5, 24) ('b', 'f', 'y') ([11], [1], [3]) baknstqzrmcxjdvygiefwoulph


In [181]:
enigma.set_wheels('a', 'a', 'a')
ct = enigma.encipher('testmessage')
assert(ct == 'olpfhnvflyn')
ct

'olpfhnvflyn'

In [182]:
enigma.set_wheels('a', 'd', 't')
ct = enigma.encipher('testmessage')
assert(ct == 'lawnjgpwjik')
ct

'lawnjgpwjik'

In [183]:
enigma.set_wheels('a', 'd', 't')
for i in range(26):
    enigma.advance()
    print('enigma.advance()')
    print("assert(enigma.wheel_positions == {})".format(enigma.wheel_positions))
    print("assert(cat(enigma.wheel_positions_l) == '{}')".format(cat(enigma.wheel_positions_l)))
    print("assert(enigma.notch_positions == {})".format(enigma.notch_positions))
    print("assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(enigma.lookup(l) for l in string.ascii_lowercase)))
    print()

enigma.advance()
assert(enigma.wheel_positions == (0, 3, 20))
assert(cat(enigma.wheel_positions_l) == 'adu')
assert(enigma.notch_positions == ([10], [25], [25]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'ehprawjbngotxikcsdqlzyfmvu')

enigma.advance()
assert(enigma.wheel_positions == (0, 3, 21))
assert(cat(enigma.wheel_positions_l) == 'adv')
assert(enigma.notch_positions == ([10], [25], [0]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'eqzxarpihmnvjkwgbfuyslodtc')

enigma.advance()
assert(enigma.wheel_positions == (0, 4, 22))
assert(cat(enigma.wheel_positions_l) == 'aew')
assert(enigma.notch_positions == ([10], [0], [1]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'qedcbtpluzmhkongavwfirsyxj')

enigma.advance()
assert(enigma.wheel_positions == (1, 5, 23))
assert(cat(enigma.wheel_positions_l) == 'bfx')
assert(enigma.notch_positions == ([11], [1], [2]))
assert(cat(enigma.lookup(l) for l in string.ascii_lowercase) == 'iwuedhs

In [184]:
enigma.set_wheels('a', 'd', 't')
ct = enigma.encipher('hellothere')
assert(ct == 'bahxvfrpdc')
ct

'bahxvfrpdc'

In [185]:
enigma.set_wheels('b', 'd', 'q')
ct = enigma.encipher('aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
assert(ct == 'kvmmwrlqlqsqpeugjrcxzwpfyiyybwloewrouvkpoztceuwtfjzqwpbqldttsr')
assert(enigma.left_wheel.position_l == 'c')
assert(enigma.middle_wheel.position_l == 'h')
assert(enigma.right_wheel.position_l == 'a')
ct

'kvmmwrlqlqsqpeugjrcxzwpfyiyybwloewrouvkpoztceuwtfjzqwpbqldttsr'

In [186]:
enigma.left_wheel.position_l

'c'

In [187]:
# Setting sheet line 31 from http://www.codesandciphers.org.uk/enigma/enigma3.htm
# Enigma simulation settings are 
# http://enigma.louisedade.co.uk/enigma.html?m3;b;b153;AFTX;AJFE;AU-BG-EY-FP-HL-IN-JZ-OS-QR-TX
w_enigma = Enigma(reflector_b_spec, 
                wheel_i_spec, wheel_i_notches,
                wheel_v_spec, wheel_v_notches,
                wheel_iii_spec, wheel_iii_notches,
                6, 20, 24,
                'ua pf rq so ni ey bg hl tx zj')

In [188]:
# # Setting sheet line 31 from http://www.codesandciphers.org.uk/enigma/enigma3.htm
# # Enigma simulation settings are 
# # http://enigma.louisedade.co.uk/enigma.html?m3;b;b153;AFTX;AJFE;AU-BG-EY-FP-HL-IN-JZ-OS-QR-TX
# enigma = Enigma(reflector_b_spec, 
#                 wheel_i_spec, wheel_i_notches,
#                 wheel_v_spec, wheel_v_notches,
#                 wheel_iii_spec, wheel_iii_notches,
#                 6, 20, 24,
#                 'ua pf rq so ni ey bg hl tx zj')

In [189]:
w_enigma.set_wheels('j', 'e', 'u')

w_enigma.advance()
assert(w_enigma.wheel_positions == (4, 11, 24))
assert(cat(w_enigma.wheel_positions_l) == 'jev')
assert(w_enigma.notch_positions == ([19], [5], [0]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'mvqjlyowkdieasgzcunxrbhtfp')

w_enigma.advance()
assert(w_enigma.wheel_positions == (4, 12, 25))
assert(cat(w_enigma.wheel_positions_l) == 'jfw')
assert(w_enigma.notch_positions == ([19], [6], [1]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'sjolzuyvrbwdpxcmtiaqfhknge')

w_enigma.advance()
assert(w_enigma.wheel_positions == (4, 12, 0))
assert(cat(w_enigma.wheel_positions_l) == 'jfx')
assert(w_enigma.notch_positions == ([19], [6], [2]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'qrxedkoywufmlvgsabpzjnicht')

w_enigma.advance()
assert(w_enigma.wheel_positions == (4, 12, 1))
assert(cat(w_enigma.wheel_positions_l) == 'jfy')
assert(w_enigma.notch_positions == ([19], [6], [3]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'hpsukliagqefwvtbjxcodnmrzy')

w_enigma.advance()
assert(w_enigma.wheel_positions == (4, 12, 2))
assert(cat(w_enigma.wheel_positions_l) == 'jfz')
assert(w_enigma.notch_positions == ([19], [6], [4]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'zevnbpyqowrtxdifhkulscjmga')


In [190]:
w_enigma.set_wheels('i', 'd', 'z')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 3))
assert(cat(w_enigma.wheel_positions_l) == 'ida')
assert(w_enigma.notch_positions == ([18], [4], [5]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'ikhpqrvcambzjondefwyxgsutl')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 4))
assert(cat(w_enigma.wheel_positions_l) == 'idb')
assert(w_enigma.notch_positions == ([18], [4], [6]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'cdabskhgzwfmlqvunyexpojtri')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 5))
assert(cat(w_enigma.wheel_positions_l) == 'idc')
assert(w_enigma.notch_positions == ([18], [4], [7]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'pcbwiqhgemyvjsuaftnroldzkx')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 6))
assert(cat(w_enigma.wheel_positions_l) == 'idd')
assert(w_enigma.notch_positions == ([18], [4], [8]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'xcbfvdnouptmlghjzwykierasq')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 7))
assert(cat(w_enigma.wheel_positions_l) == 'ide')
assert(w_enigma.notch_positions == ([18], [4], [9]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'xfvglbdynuseriwqpmkzjcoaht')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 8))
assert(cat(w_enigma.wheel_positions_l) == 'idf')
assert(w_enigma.notch_positions == ([18], [4], [10]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'tfpqlbouynsewjgcdxkahzmriv')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 9))
assert(cat(w_enigma.wheel_positions_l) == 'idg')
assert(w_enigma.notch_positions == ([18], [4], [11]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'cjaunvlwtbygzexrspqidfhokm')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 10))
assert(cat(w_enigma.wheel_positions_l) == 'idh')
assert(w_enigma.notch_positions == ([18], [4], [12]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'yltxkrqvowebzpingfucshjdam')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 11))
assert(cat(w_enigma.wheel_positions_l) == 'idi')
assert(w_enigma.notch_positions == ([18], [4], [13]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'myktluzrnxceaiqsohpdfwvjbg')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 12))
assert(cat(w_enigma.wheel_positions_l) == 'idj')
assert(w_enigma.notch_positions == ([18], [4], [14]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'pynjrmiugdqxfcvakewzhoslbt')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 13))
assert(cat(w_enigma.wheel_positions_l) == 'idk')
assert(w_enigma.notch_positions == ([18], [4], [15]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'mwvedyplnoxhaijgrqtszcbkfu')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 14))
assert(cat(w_enigma.wheel_positions_l) == 'idl')
assert(w_enigma.notch_positions == ([18], [4], [16]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'qcbrfeutvoxpnmjladzhgiykws')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 15))
assert(cat(w_enigma.wheel_positions_l) == 'idm')
assert(w_enigma.notch_positions == ([18], [4], [17]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'dnoahryetsmukbcvwfjilpqzgx')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 16))
assert(cat(w_enigma.wheel_positions_l) == 'idn')
assert(w_enigma.notch_positions == ([18], [4], [18]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'nidcfehgbqsovalyjzkxwmutpr')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 17))
assert(cat(w_enigma.wheel_positions_l) == 'ido')
assert(w_enigma.notch_positions == ([18], [4], [19]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'joifxdulcarhzpbntkwqgysevm')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 18))
assert(cat(w_enigma.wheel_positions_l) == 'idp')
assert(w_enigma.notch_positions == ([18], [4], [20]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'ptnlsxvozmwdjchayuebrgkfqi')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 19))
assert(cat(w_enigma.wheel_positions_l) == 'idq')
assert(w_enigma.notch_positions == ([18], [4], [21]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'slwopzqnmxybihdeguavrtcjkf')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 20))
assert(cat(w_enigma.wheel_positions_l) == 'idr')
assert(w_enigma.notch_positions == ([18], [4], [22]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'hcbedwlamzogixkytsrqvufnpj')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 21))
assert(cat(w_enigma.wheel_positions_l) == 'ids')
assert(w_enigma.notch_positions == ([18], [4], [23]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'odxbjwzrmelkisavuhnyqpfctg')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 22))
assert(cat(w_enigma.wheel_positions_l) == 'idt')
assert(w_enigma.notch_positions == ([18], [4], [24]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'udgbfeclrwnhxksvtioqapjmzy')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 23))
assert(cat(w_enigma.wheel_positions_l) == 'idu')
assert(w_enigma.notch_positions == ([18], [4], [25]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'nrdczqxmowvshaiufblypkjgte')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 24))
assert(cat(w_enigma.wheel_positions_l) == 'idv')
assert(w_enigma.notch_positions == ([18], [4], [0]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'hkifjdoacebqtzgulyvmpsxwrn')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 11, 25))
assert(cat(w_enigma.wheel_positions_l) == 'iew')
assert(w_enigma.notch_positions == ([18], [5], [1]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'yptzuhofqvnmlkgbixwcejsrad')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 11, 0))
assert(cat(w_enigma.wheel_positions_l) == 'iex')
assert(w_enigma.notch_positions == ([18], [5], [2]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'vkdcwhqfjibzsptngumoraeyxl')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 11, 1))
assert(cat(w_enigma.wheel_positions_l) == 'iey')
assert(w_enigma.notch_positions == ([18], [5], [3]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'wenpbqrouxlkychdfgzvitajms')


In [191]:
w_enigma.notch_positions

([18], [5], [3])

In [192]:
w_enigma.set_wheels('i', 'd', 'z')
ct = w_enigma.encipher('verylongtestmessagewithanextrabitofmessageforgoodmeasure')
assert(ct == 'gstsegeqdrthkfwesljjomfvcqwcfspxpfqqmewvddybarzwubxtpejz')
assert(w_enigma.wheel_positions == (3, 12, 6))
assert(cat(w_enigma.wheel_positions_l) == 'ifd')
assert(w_enigma.notch_positions == ([18], [6], [8]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'urygzpdmxtwshqvfnbljaokice')

w_enigma.set_wheels('i', 'd', 'z')
pt = w_enigma.encipher('gstsegeqdrthkfwesljjomfvcqwcfspxpfqqmewvddybarzwubxtpejz')
assert(pt == 'verylongtestmessagewithanextrabitofmessageforgoodmeasure')

pt, w_enigma.wheel_positions, w_enigma.wheel_positions_l, w_enigma.notch_positions, cat(w_enigma.lookup(l) for l in string.ascii_lowercase)

('verylongtestmessagewithanextrabitofmessageforgoodmeasure',
 (3, 12, 6),
 ('i', 'f', 'd'),
 ([18], [6], [8]),
 'urygzpdmxtwshqvfnbljaokice')

In [193]:
w_enigma.set_wheels('i', 'd', 'z')

for i in range(26):
    w_enigma.advance()
    print('w_enigma.advance()')
    print("assert(w_enigma.wheel_positions == {})".format(w_enigma.wheel_positions))
    print("assert(cat(w_enigma.wheel_positions_l) == '{}')".format(cat(w_enigma.wheel_positions_l)))
    print("assert(w_enigma.notch_positions == {})".format(w_enigma.notch_positions))
    print("assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(w_enigma.lookup(l) for l in string.ascii_lowercase)))
    print()

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 3))
assert(cat(w_enigma.wheel_positions_l) == 'ida')
assert(w_enigma.notch_positions == ([18], [4], [5]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'ikhpqrvcambzjondefwyxgsutl')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 4))
assert(cat(w_enigma.wheel_positions_l) == 'idb')
assert(w_enigma.notch_positions == ([18], [4], [6]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'cdabskhgzwfmlqvunyexpojtri')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 5))
assert(cat(w_enigma.wheel_positions_l) == 'idc')
assert(w_enigma.notch_positions == ([18], [4], [7]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'pcbwiqhgemyvjsuaftnroldzkx')

w_enigma.advance()
assert(w_enigma.wheel_positions == (3, 10, 6))
assert(cat(w_enigma.wheel_positions_l) == 'idd')
assert(w_enigma.notch_positions == ([18], [4], [8]))
assert(cat(w_enigma.lookup(l) for l i

In [215]:
w_enigma.set_wheels('a', 'y', 't')

print("self.assertEqual(self.enigma31.wheel_positions, {})".format(w_enigma.wheel_positions))
print("self.assertEqual(cat(self.enigma31.wheel_positions_l), '{}')".format(cat(w_enigma.wheel_positions_l)))
print("self.assertEqual(self.enigma31.notch_positions, {})".format(w_enigma.notch_positions))
print("assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(w_enigma.lookup(l) for l in string.ascii_lowercase)))
print()

for i in range(5):
    w_enigma.advance()
    print('self.enigma31.advance()')
    print("self.assertEqual(self.enigma31.wheel_positions, {})".format(w_enigma.wheel_positions))
    print("self.assertEqual(cat(self.enigma31.wheel_positions_l), '{}')".format(cat(w_enigma.wheel_positions_l)))
    print("self.assertEqual(self.enigma31.notch_positions, {})".format(w_enigma.notch_positions))
    print("assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(w_enigma.lookup(l) for l in string.ascii_lowercase)))
    print()

self.assertEqual(self.enigma31.wheel_positions, (21, 5, 22))
self.assertEqual(cat(self.enigma31.wheel_positions_l), 'ayt')
self.assertEqual(self.enigma31.notch_positions, ([10], [25], [24]))
assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == 'izhrgtecaslkywvqpdjfxonumb')

self.enigma31.advance()
self.assertEqual(self.enigma31.wheel_positions, (21, 5, 23))
self.assertEqual(cat(self.enigma31.wheel_positions_l), 'ayu')
self.assertEqual(self.enigma31.notch_positions, ([10], [25], [25]))
assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == 'dtoavihgflwjnmcsrqpbzekyxu')

self.enigma31.advance()
self.assertEqual(self.enigma31.wheel_positions, (21, 5, 24))
self.assertEqual(cat(self.enigma31.wheel_positions_l), 'ayv')
self.assertEqual(self.enigma31.notch_positions, ([10], [25], [0]))
assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == 'xquhtpsdwkjonmlfbvgecriazy')

self.enigma31.advance()
self.assertEqual(self.enigma31.wheel_positions, (2

In [216]:
w_enigma.set_wheels('a', 'z', 't')

print("self.assertEqual(self.enigma31.wheel_positions, {})".format(w_enigma.wheel_positions))
print("self.assertEqual(cat(self.enigma31.wheel_positions_l), '{}')".format(cat(w_enigma.wheel_positions_l)))
print("self.assertEqual(self.enigma31.notch_positions, {})".format(w_enigma.notch_positions))
print("assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(w_enigma.lookup(l) for l in string.ascii_lowercase)))
print()

for i in range(5):
    w_enigma.advance()
    print('self.enigma31.advance()')
    print("self.assertEqual(self.enigma31.wheel_positions, {})".format(w_enigma.wheel_positions))
    print("self.assertEqual(cat(self.enigma31.wheel_positions_l), '{}')".format(cat(w_enigma.wheel_positions_l)))
    print("self.assertEqual(self.enigma31.notch_positions, {})".format(w_enigma.notch_positions))
    print("assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == '{}')".format(cat(w_enigma.lookup(l) for l in string.ascii_lowercase)))
    print()

self.assertEqual(self.enigma31.wheel_positions, (21, 6, 22))
self.assertEqual(cat(self.enigma31.wheel_positions_l), 'azt')
self.assertEqual(self.enigma31.notch_positions, ([10], [0], [24]))
assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == 'idjbptqwacsvnmregokfzlhyxu')

self.enigma31.advance()
self.assertEqual(self.enigma31.wheel_positions, (22, 7, 23))
self.assertEqual(cat(self.enigma31.wheel_positions_l), 'bau')
self.assertEqual(self.enigma31.notch_positions, ([11], [1], [25]))
assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == 'rniszouwcxtvqbfymadkglhjpe')

self.enigma31.advance()
self.assertEqual(self.enigma31.wheel_positions, (22, 7, 24))
self.assertEqual(cat(self.enigma31.wheel_positions_l), 'bav')
self.assertEqual(self.enigma31.notch_positions, ([11], [1], [0]))
assert(cat(self.enigma31.lookup(l) for l in string.ascii_lowercase) == 'qijfsdmkbchugxtwazeolypnvr')

self.enigma31.advance()
self.assertEqual(self.enigma31.wheel_positions, (22, 

In [221]:
w_enigma.set_wheels('i', 'z', 'd')
ct = w_enigma.encipher('verylongtestmessagewithanextrabitofmessageforgoodmeasure')
assert(ct == 'apocwtjuikurcfivlozvhffkoacxufcekthcvodfqpxdjqyckdozlqki')
assert(w_enigma.wheel_positions == (4, 9, 10))
assert(cat(w_enigma.wheel_positions_l) == 'jch')
assert(w_enigma.notch_positions == ([19], [3], [12]))
assert(cat(w_enigma.lookup(l) for l in string.ascii_lowercase) == 'mopnigfuesqwadbcktjrhylzvx')

w_enigma.set_wheels('i', 'z', 'd')
pt = w_enigma.decipher('apocwtjuikurcfivlozvhffkoacxufcekthcvodfqpxdjqyckdozlqki')
assert(pt == 'verylongtestmessagewithanextrabitofmessageforgoodmeasure')

In [218]:
w_enigma.notch_positions

([19], [3], [12])

In [194]:
# Reflector B
# Rotors III, I, II with rings 17, 11, 19
# Plugboard pairs GU FZ BD LK TC PS HV WN JE AM

tbt_enigma = Enigma(reflector_b_spec, 
                wheel_iii_spec, wheel_iii_notches,
                wheel_i_spec, wheel_i_notches,
                wheel_ii_spec, wheel_ii_notches,
                17, 11, 19,
                'GU FZ BD LK TC PS HV WN JE AM'.lower())

In [195]:
tbt_enigma.set_wheels('a', 'q', 'v')
ct = tbt_enigma.encipher('very long test message with an extra bit of message for good measure')
ct

'jbvbwwzfslhxnhzzccsngebmrnswgjonwbjnzcfgadeuoyameylmpvny'

In [196]:
target_ct = 'SLGNC SZXLT KZEBG HSTGY WDMPR'
target_ct

'SLGNC SZXLT KZEBG HSTGY WDMPR'

In [197]:
target_pt = 'Theyw erede tecte d byBri tishs hipsi nclud'
target_pt

'Theyw erede tecte d byBri tishs hipsi nclud'

In [198]:
len(target_ct), len(target_pt)

(29, 43)

In [199]:
print('{}\n{}'.format(target_ct, target_pt))

SLGNC SZXLT KZEBG HSTGY WDMPR
Theyw erede tecte d byBri tishs hipsi nclud


In [200]:
tbt_enigma.set_wheels('a', 'a', 'a')
this_pt = tbt_enigma.encipher(target_ct)

tbt_enigma.set_wheels('a', 'a', 'a')
this_ct = tbt_enigma.encipher(target_pt)


print('{}\n{}\n{}\n{}'.format(target_pt, target_ct, this_ct, this_pt))

Theyw erede tecte d byBri tishs hipsi nclud
SLGNC SZXLT KZEBG HSTGY WDMPR
slgncszxltkzebghstgywdmprucuzqdqzpve
theyweredetectedbybritish


In [201]:
import itertools

In [202]:
def str_ham(s1, s2):
    """Hamming distance for strings"""
    return sum(1 for c1, c2 in zip(s1, s2) if c1 != c2)

In [203]:
str_ham('hello', 'hello')

0

A brute-force check of all message settings, looking for the one that generates the target text.

In [204]:
%%timeit
best = ('a', 'a', 'a')
best_hd = 10000
for w1, w2, w3 in itertools.product(string.ascii_lowercase, repeat=3):
    tbt_enigma.set_wheels(w1, w2, w3)
    this_ct = tbt_enigma.encipher(target_pt)
    if this_ct == target_ct:
        print(w1, w2, w3)
    if str_ham(this_ct, target_ct) < best_hd:
        best = (w1, w2, w3)
        best_hd = str_ham(this_ct, target_ct)
print('best', best, best_hd)

best ('a', 'a', 'a') 29
best ('a', 'a', 'a') 29
best ('a', 'a', 'a') 29
best ('a', 'a', 'a') 29
1 loop, best of 3: 20.6 s per loop


In [205]:
tbt_enigma.set_wheels('a', 'a', 'a')
tbt_enigma.encipher(target_pt, debug=True), target_ct

Wheels now ['a', 'a', 'b'] enciphering t -> s
Wheels now ['a', 'a', 'c'] enciphering h -> l
Wheels now ['a', 'a', 'd'] enciphering e -> g
Wheels now ['a', 'a', 'e'] enciphering y -> n
Wheels now ['a', 'b', 'f'] enciphering w -> c
Wheels now ['a', 'b', 'g'] enciphering e -> s
Wheels now ['a', 'b', 'h'] enciphering r -> z
Wheels now ['a', 'b', 'i'] enciphering e -> x
Wheels now ['a', 'b', 'j'] enciphering d -> l
Wheels now ['a', 'b', 'k'] enciphering e -> t
Wheels now ['a', 'b', 'l'] enciphering t -> k
Wheels now ['a', 'b', 'm'] enciphering e -> z
Wheels now ['a', 'b', 'n'] enciphering c -> e
Wheels now ['a', 'b', 'o'] enciphering t -> b
Wheels now ['a', 'b', 'p'] enciphering e -> g
Wheels now ['a', 'b', 'q'] enciphering d -> h
Wheels now ['a', 'b', 'r'] enciphering b -> s
Wheels now ['a', 'b', 's'] enciphering y -> t
Wheels now ['a', 'b', 't'] enciphering b -> g
Wheels now ['a', 'b', 'u'] enciphering r -> y
Wheels now ['a', 'b', 'v'] enciphering i -> w
Wheels now ['a', 'b', 'w'] enciphe

('slgncszxltkzebghstgywdmprucuzqdqzpve', 'SLGNC SZXLT KZEBG HSTGY WDMPR')

In [206]:
tbt_enigma.set_wheels('a', 'a', 'a')
tbt_enigma.encipher(target_ct, debug=True), target_pt

Wheels now ['a', 'a', 'b'] enciphering s -> t
Wheels now ['a', 'a', 'c'] enciphering l -> h
Wheels now ['a', 'a', 'd'] enciphering g -> e
Wheels now ['a', 'a', 'e'] enciphering n -> y
Wheels now ['a', 'b', 'f'] enciphering c -> w
Wheels now ['a', 'b', 'g'] enciphering s -> e
Wheels now ['a', 'b', 'h'] enciphering z -> r
Wheels now ['a', 'b', 'i'] enciphering x -> e
Wheels now ['a', 'b', 'j'] enciphering l -> d
Wheels now ['a', 'b', 'k'] enciphering t -> e
Wheels now ['a', 'b', 'l'] enciphering k -> t
Wheels now ['a', 'b', 'm'] enciphering z -> e
Wheels now ['a', 'b', 'n'] enciphering e -> c
Wheels now ['a', 'b', 'o'] enciphering b -> t
Wheels now ['a', 'b', 'p'] enciphering g -> e
Wheels now ['a', 'b', 'q'] enciphering h -> d
Wheels now ['a', 'b', 'r'] enciphering s -> b
Wheels now ['a', 'b', 's'] enciphering t -> y
Wheels now ['a', 'b', 't'] enciphering g -> b
Wheels now ['a', 'b', 'u'] enciphering y -> r
Wheels now ['a', 'b', 'v'] enciphering w -> i
Wheels now ['a', 'b', 'w'] enciphe

('theyweredetectedbybritish', 'Theyw erede tecte d byBri tishs hipsi nclud')

In [207]:
tbt_enigma.set_wheels('a', 'a', 'a')
tbt_enigma.encipher(target_ct), target_pt

('theyweredetectedbybritish', 'Theyw erede tecte d byBri tishs hipsi nclud')

In [208]:
cat(tbt_enigma.plugboard.forward(l) for l in string.ascii_lowercase)

'mdtbjzuvielkawosqrpcghnxyf'

In [209]:
tbt_enigma.set_wheels('a', 'a', 'a')
tbt_enigma.left_wheel.position

10